<a href="https://colab.research.google.com/github/sergioGarcia91/ML_Carolina_Bays/blob/main/06_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, **30 Random Forest Classifier models** will be trained, following a similar approach to logistic regression. Since category 0 contains more pixels than category 1, **downsampling** will be applied to category 0 to balance the dataset.  

The process involves iteratively separating the data from both categories. In each iteration, the number of samples in category 1 will be counted, and an equal number of samples from category 0 will be randomly selected.  

To introduce more randomness during training, a new **train-test split** will be performed in each iteration, ensuring that the training data for category 1 varies in every cycle.  



# Start

In [ ]:
!pip install tables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.9/398.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.1/506.1 kB 36.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import h5py
import multiprocessing
import joblib

from IPython.display import clear_output
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
path_saveCSV = '/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/Dataset_CSV'

df = pd.read_hdf(os.path.join(path_saveCSV, 'TRAIN_CarolinaBays_AOI_01_03.h5'), 'df')

df.head()

,B1,B2,B3,B4,B5,B6,B7,B2_B1,B3_B1,B4_B1,...,B5_B3,B6_B3,B7_B3,B5_B4,B6_B4,B7_B4,B6_B5,B7_B5,B7_B6,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Total of data 103327744
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103327744 entries, 0 to 103327743
Data columns (total 29 columns):
 #   Column  Dtype  
---  ------  -----  
 0   B1      float32
 1   B2      float32
 2   B3      float32
 3   B4      float32
 4   B5      float32
 5   B6      float32
 6   B7      float32
 7   B2_B1   float32
 8   B3_B1   float32
 9   B4_B1   float32
 10  B5_B1   float32
 11  B6_B1   float32
 12  B7_B1   float32
 13  B3_B2   float32
 14  B4_B2   float32
 15  B5_B2   float32
 16  B6_B2   float32
 17  B7_B2   float32
 18  B4_B3   float32
 19  B5_B3   float32
 20  B6_B3   float32
 21  B7_B3   float32
 22  B5_B4   float32
 23  B6_B4   float32
 24  B7_B4   float32
 25  B6_B5   float32
 26  B7_B5   float32
 27  B7_B6   float32
 28  y       float32
dtypes: float32(29)
memory usage: 11.9 GB


# Split and training

In [ ]:
num_cores = multiprocessing.cpu_count()
print(f"Number of available cores: {num_cores}")


Number of available cores: 96


In [ ]:
path_save_models = '/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/ML_models/'


- A test was conducted with 100 estimators or trees, and it took about 20 minutes per iteration, with a score of 0.6. The number of estimators has been reduced to 40 to avoid long processing times and to be able to generate the 30 models.

- 40 estimators were chosen because when the test was conducted with 100, it first trained 42 and then the remaining ones. The process took 12 minutes, and the training score remained at 0.6. Now, 30 trees will be tested to reduce the training time.

- With 30 estimators, the process took 8 minutes, and the score is still 0.6. Now, the depth will be reduced to 5 and the test will be conducted again to improve the training time and possibly the score.

- It took 7 minutes with a depth of 5, and the score remained at 0.6. The depth will be kept at 10, as no significant improvement in time was observed.



In [ ]:
print_text = True
print_text_Training = True
verbose_print = True

count_models = 21 # Indicate the model number that will be saved
# If the process was stopped and you want to continue from the previous amount,
# you should specify the number from which you want to start

total_models = count_models + 10

target_score = 0.6 # In the tests, it never exceeded a score of 0.6

count_trial = 1

train_score_list = []
test_score_list = []
models_name_list = []
elapsed_time_list = []
trial_list = []


while count_models < total_models:
  # Start the timer
  start_time = time.time() # Each iteration takes less than 10 minutes

  clear_output(wait=True)

  # Create empty DataFrames for train and test
  df_train = pd.DataFrame()
  df_test = pd.DataFrame()

  # Filter data
  category_data_1 = df[df['y'] == 1].copy().reset_index(drop=True)
  category_data_0 = df[df['y'] == 0].copy().reset_index(drop=True)

  # Calculate 80% for train and 20% for test
  train_size = int(0.8 * len(category_data_1))
  test_size = len(category_data_1) - train_size

  # Select randomly to shuffle the data
  category_data_1 = category_data_1.sample(frac=1).reset_index(drop=True)
  category_data_0 = category_data_0.sample(frac=1).reset_index(drop=True)

  # Split into train and test
  category_train_1 = category_data_1[:train_size]
  category_train_0 = category_data_0[:train_size]
  category_test_1 = category_data_1[train_size:]
  category_test_0 = category_data_0[train_size:]

  category_train = pd.concat([category_train_1, category_train_0], ignore_index=True)
  category_train = category_train.sample(frac=1).reset_index(drop=True)
  category_test = pd.concat([category_test_1, category_test_0], ignore_index=True)
  category_test = category_test.sample(frac=1).reset_index(drop=True)

  if print_text:
    print(f'Train size: {len(category_train_1)*2}')
    print(f'Test size: {len(category_test_1)*2}')
    print('---'*3)

  # Concatenate the data into the corresponding DataFrames
  df_train = pd.concat([df_train, category_train], ignore_index=True)
  df_test = pd.concat([df_test, category_test], ignore_index=True)
  if print_text:
    print(f'DF Train size: {df_train.shape[0]}')
    print(f'DF Test size: {df_test.shape[0]}')
    print('\n')

  # Datos to Train and Test
  X_train = df_train.iloc[:, :-1].to_numpy()
  y_train = df_train['y'].to_numpy()

  X_test = df_test.iloc[:, :-1].to_numpy()
  y_test = df_test['y'].to_numpy()

  if print_text:
    print('Shapes X_train, y_train, X_test, y_test')
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

  # Create the model
  model_RandForest = RandomForestClassifier(n_estimators= 30, # 50
                                            max_depth= 10, # 7
                                            min_samples_split= 0.05, # 100
                                            min_samples_leaf= 0.01, # 200
                                            bootstrap= True, # False
                                            max_samples= 0.7,
                                            verbose= verbose_print,
                                            n_jobs= -1) # -1 means using all processors

  # Train the model
  print('---'*10)
  print(f'Trial: {count_trial}')
  model_RandForest.fit(X_train, y_train)

  # End the timer
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time

  train_score = model_RandForest.score(X_train, y_train)
  test_score = model_RandForest.score(X_test, y_test)

  if train_score > target_score:
    if print_text_Training:
      print(f'Train score: {train_score:.4f}')
      print(f'Test score: {test_score:.4f}')
      print(f'Elapsed time: {elapsed_time:.2f} seconds')
      print('\n')

    # Save model
    if count_models < 10:
      Name = f'model_RandForest_00{count_models}.pkl'
    elif count_models < 100:
      Name = f'model_RandForest_0{count_models}.pkl'
    else:
      Name = f'model_RandForest_{count_models}.pkl'

    joblib.dump(model_RandForest, path_save_models + Name)
    print(f'---> Model saved as {Name}')
    print('\n')

    train_score_list.append(train_score)
    test_score_list.append(test_score)
    models_name_list.append(Name)
    elapsed_time_list.append(round(elapsed_time, 2))
    trial_list.append(count_trial)

    count_models += 1

  else:
    print(f'Train score: {train_score:.4f}')
    print(f'Elapsed time: {elapsed_time:.2f} seconds')
    print('No model was generated.')
    print('\n')

  count_trial += 1


Train size: 18602534
Test size: 4650634
---------
DF Train size: 18602534
DF Test size: 84725210


Shapes X_train, y_train, X_test, y_test
(18602534, 28) (18602534,) (84725210, 28) (84725210,)
------------------------------
Trial: 10


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    7.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   35.2s finished


Train score: 0.6568
Test score: 0.6104
Elapsed time: 484.27 seconds


---> Model saved as model_RandForest_030.pkl




# Df models

In [ ]:
dict_model = {'Trial': trial_list,
              'Model': models_name_list,
              'Train score': train_score_list,
              'Test score': test_score_list,
              'Elapsed time': elapsed_time_list} # Total time per iteration

df_models = pd.DataFrame(dict_model)

df_models

,Trial,Model,Train score,Test score,Elapsed time
0,1,model_RandForest_021.pkl,0.655732,0.610209,478.90
1,2,model_RandForest_022.pkl,0.657562,0.614252,481.62
2,3,model_RandForest_023.pkl,0.655046,0.620079,472.78
3,4,model_RandForest_024.pkl,0.656639,0.601629,483.11
4,5,model_RandForest_025.pkl,0.657693,0.609228,487.16
5,6,model_RandForest_026.pkl,0.655960,0.600129,479.89
6,7,model_RandForest_027.pkl,0.654347,0.613994,480.52
7,8,model_RandForest_028.pkl,0.656661,0.603452,486.45
8,9,model_RandForest_029.pkl,0.657011,0.610493,477.03
9,10,model_RandForest_030.pkl,0.656755,0.610435,484.27


In [ ]:
df_models.describe().round(2)

,Trial,Train score,Test score,Elapsed time
count,10.00,10.00,10.00,10.00
mean,5.50,0.66,0.61,481.17
std,3.03,0.00,0.01,4.37
min,1.00,0.65,0.60,472.78
25%,3.25,0.66,0.60,479.15
50%,5.50,0.66,0.61,481.07
75%,7.75,0.66,0.61,483.98
max,10.00,0.66,0.62,487.16


## Save Df

In [ ]:
df_models.to_csv(path_save_models + 'df_30models_RandForest.csv',
                 sep=';',
                 decimal=',',
                 index=False)

# End